<a href="https://colab.research.google.com/github/Nishidh25/Covid-19-Detection-from-Lung-X-rays/blob/master/Covid_19_Detection_from_Lung_X_rays_Web_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Front end website for [Covid-19-Detection-from-Lung-X-rays ](https://github.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays) 

Takes an image and classifies it as Covid-19+Pneumonia

[Notebook - 1](https://github.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/blob/master/Covid_19_Detection_from_Lung_X_rays.ipynb)

[Notebook - 2](https://github.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/blob/master/Covid_19_Detection_from_Lung_X_rays_Web_App.ipynb) (Current)

# Downloading model and required html files from github

In [1]:
# Creating directories for flask
!mkdir /content/static 
!mkdir /content/templates

# Downloading resources from github 
!wget https://github.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/raw/master/static/covid_positive.jpeg
!wget https://github.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/raw/master/static/normal.jpeg
!wget https://raw.githubusercontent.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/master/templates/image_predicted.html
!wget https://raw.githubusercontent.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/master/templates/main.html
!wget https://github.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/raw/master/model.h5 # Trained Model

#########################################################################################################
# Model uploaded on Github lfs wont download once bandwidth is over you will get an error while running #
# Download and upload model manually                                                                    # 
#########################################################################################################

# Copy the files to the respective directories
!cp covid_positive.jpeg /content/static
!cp normal.jpeg /content/static
!cp main.html /content/templates
!cp image_predicted.html /content/templates

--2020-06-11 14:06:28--  https://github.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/raw/master/static/covid_positive.jpeg
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/master/static/covid_positive.jpeg [following]
--2020-06-11 14:06:28--  https://raw.githubusercontent.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/master/static/covid_positive.jpeg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114594 (112K) [image/jpeg]
Saving to: ‘covid_positive.jpeg’

covid_positive.jpeg 100%[===================>] 111.91K  --.-KB/s    in 0.04s   

2020-06-11

# Imports

In [2]:
!pip install flask_ngrok
import os
import numpy as np
import tensorflow as tf
from flask_ngrok import run_with_ngrok
from werkzeug.utils import secure_filename
from flask import Flask, request, render_template, flash, redirect, url_for

# Prediction function



In [0]:
def predict(file):  
  """
  Takes file directory as input , loads and pre processes the image and inputs 
  it to the model 

  Input : file(str)
  Output : class(string) , pred(float)
  """
  img = tf.io.read_file(file) # Read file from directory
  img = tf.image.decode_jpeg(img, channels=3) # Decode it, RGB 3 channels
  img.set_shape([None, None, 3]) # Clearing the shape of image
  img = tf.image.resize(img, (224, 224)) # reshaping the image to 224, 224
  img = np.array(img) # convert to numpy array
  img = np.expand_dims(img, 0) # make 'batch' of 1 , input type of model now [1,224,224,3]
  img = img /255. # Rescaling 
  model =  tf.keras.models.load_model("model.h5") # load model 
  pred = model.predict(img,verbose=1) # Predict image
  if pred > 0.5 :
    return "Classified as Covid-19 Positive",pred
  else : return "Classified as Covid-19 Negative",pred

# Flask api
Run the below code and select the second link (*.ngrok.io) to view the site.

**If you get any error while running , please check the downloaded files**

In [7]:
UPLOAD_FOLDER = '/content/static' # Upload folder
ALLOWED_EXTENSIONS = {'jpg', 'jpeg'} # File type accepted

app = Flask(__name__) 
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
run_with_ngrok(app)

def allowed_file(filename): 
  """
  Check if the file is a jpg or jpeg and returns it 
  """
  return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            return render_template('main.html')
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            return render_template('main.html')
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            image_dir = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            pred,acc = predict(image_dir)
            acc = "{:.2f}".format(acc[0][0] * 100)
            return render_template("image_predicted.html", user_image = file.filename, prediction = pred , percent_accuracy = acc)
    return render_template('main.html')
if __name__ == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://76fa9dce82e5.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [11/Jun/2020 14:40:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2020 14:40:41] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 2ms/step


127.0.0.1 - - [11/Jun/2020 14:40:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2020 14:40:48] "GET /static/covid_positive.jpeg HTTP/1.1" 200 -


[Notebook - 1](https://github.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/blob/master/Covid_19_Detection_from_Lung_X_rays.ipynb)

[Notebook - 2](https://github.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays/blob/master/Covid_19_Detection_from_Lung_X_rays_Web_App.ipynb) (Current)

[Github Project](https://github.com/Nishidh25/Covid-19-Detection-from-Lung-X-rays)